# Práctica 1 de Procesado de Lenguaje Natural:
# Uso de expresiones regulares en Python
En esta práctica vamos a utilizar el módulo `re` de Python y la librería Pandas para tratar con expresiones regulares. 


### Nombres:
Introduce en esta celda los nombres de los dos integrantes del grupo:\
Daniel Lillo Plaza

## Búsqueda de patrones
En el módulo `re` hay 4 métodos para buscar un patrón de texto: `re.match`, `re.search`, `re.findall` y `re.finditer`

In [1]:
import re

texto = '"Ethics are built right into the ideals and objectives of the United Nations" \
#UNSG @NY @UN_Women Society for Ethical Culture bit.ly/2guVelr'

print(re.match(r'@[A-Z]+', texto))

None


`re.match` sólo busca al inicio de la cadena, `re.search` busca la primera aparición del patrón en el texto.

In [2]:
print(re.match(r'#[A-Z]+', texto))

None


In [3]:
print(re.search(r'@[\w]+', texto))

<re.Match object; span=(84, 87), match='@NY'>


In [4]:
print(re.search(r'@[A-Z]+', texto))

<re.Match object; span=(84, 87), match='@NY'>


`re.match` y `re.search` devuelven un objeto `Match` que tiene varios métodos. El método `group()` contiene el patrón encontrado

In [5]:
match = re.search(r'#[A-Z]+', texto)
print('span:',match.span())
print('start:',match.start())
print('end:',match.end())
print('group:',match.group())

span: (78, 83)
start: 78
end: 83
group: #UNSG


Para buscar todas las apariciones de un patrón en el texto usamos la función `re.findall`

In [6]:
print(re.findall(r'@[\w_]+', texto))

['@NY', '@UN_Women']


## Patrones complejos
Ejemplo: queremos detectar cualquier fecha en un texto, pero la fecha puede seguir distintos patrones. Hay que definir un patrón con todas las posibilidades...

In [7]:
fechas = '23/9/2010, 23/09/2010, 23/9-10, 23-9-2010, 2/9/2010'

In [8]:
re.findall(r'\d{2}/\d{1,2}/\d{4}', fechas) #captura sólo las fechas con formado dd/(m)m/aaaa

['23/9/2010', '23/09/2010']

In [9]:
re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', fechas) #captura todos los formatos de fecha posibles

['23/9/2010', '23/09/2010', '23/9-10', '23-9-2010', '2/9/2010']

In [10]:
texto = 'El avión salió el 5/10/12 de Caracas'
re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', texto)

['5/10/12']

### Ejercicio 1
¿Cómo podemos detectar *todas* las fechas de este texto?  
*Ayuda: Utiliza el operador OR* (`|`) *para especificar dos patrones alternativos*

In [11]:
texto = '''Francisco nació el 28/3/78, se casó el 20 de mayo del 98 y tuvo 2 hijos,
el primero nació el 3-10-2001 y el segundo el 2 de junio de 2004'''

In [12]:
# Solución
re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{1,2} [a-z]{2} [a-z]{4,10} [a-z]{2,3} \d{2,4}', texto)

['28/3/78', '20 de mayo del 98', '3-10-2001', '2 de junio de 2004']

## Captura de grupos
Podemos buscar un patrón compuesto pero sólo recuperar una parte específica de éste.

In [13]:
texto = '''Ingredientes de Arroz con Leche:
- 200 gramos de arroz
- 150 gramos de azúcar
- Un litro de leche entera
- Dos ramas de canela
- Piel de un limón
- Canela molida
- 50 gramos de mantequilla (Opcional)
'''

*¿Cuántos gr de azúcar tiene la receta?*  
Podemos buscar todos los números del texto:

In [14]:
re.findall(r'\d+', texto)

['200', '150', '50']

Podemos buscar el patrón de los gramos de azúcar

In [15]:
re.findall(r'\d+ gramos de azúcar', texto)

['150 gramos de azúcar']

Si de este patrón queremos devolver sólo la parte correspondiente al número lo asignamos a un grupo:

In [16]:
re.findall(r'(\d+) gramos de azúcar', texto)

['150']

Podemos definir varios grupos en la captura y recuperarlos como una tupla de textos.  
*P.ej. ¿Cuántos gramos de cada ingrediente tenemos?*

In [17]:
re.findall(r'(\d+) gramos de (\w+)', texto)

[('200', 'arroz'), ('150', 'azúcar'), ('50', 'mantequilla')]

Incluso podemos asignar un nombre a cada grupo. El objeto devuelto crea un diccionario con los grupos encontrados. 

In [18]:
matches = re.search(r'(?P<gramos>\d+) gramos de (?P<ingrediente>\w+)', texto)
print(matches.groups())
matches.groupdict()

('200', 'arroz')


{'gramos': '200', 'ingrediente': 'arroz'}

In [19]:
matches.group('gramos') #alternativamente matches['gramos']

'200'

In [20]:
matches.group('ingrediente') #alternativamente matches['ingrediente']

'arroz'

Para buscar todas las repeticiones de un patrón con grupos enumerados no podemos usar `re.findall` sino que tenemos que usar la versión iterativa `re.finditer` que devuelve un *iterable* sobre objetos de tipo `re.Match`

In [21]:
matches = list(re.finditer(r'(?P<gramos>\d+) gramos de (?P<ingrediente>\w+)', texto))
matches

[<re.Match object; span=(35, 54), match='200 gramos de arroz'>,
 <re.Match object; span=(57, 77), match='150 gramos de azúcar'>,
 <re.Match object; span=(164, 188), match='50 gramos de mantequilla'>]

In [22]:
matches[1].groupdict()

{'gramos': '150', 'ingrediente': 'azúcar'}

Podemos crear una *list comprehension* recorriendo sobre el resultado de `re.findall`:

In [23]:
[(ingredientes, gramos) for (gramos, ingredientes) in re.findall(r'(?P<gramos>\d+) gramos de (?P<ingrediente>\w+)', texto)]

[('arroz', '200'), ('azúcar', '150'), ('mantequilla', '50')]

### Ejercicio 2
Genera un diccionario de Python con los gramos de cada ingrediente a partir de la lista `matches`.  
El resultado debe ser:  
```python
{'arroz': '200', 'azúcar': '150', 'mantequilla': '50'}
```

In [24]:
#Solución

diccionario_ingredientes = {}

for ingrediente in [(matches[i].groupdict()) for i in (range(len(matches)))]:
    diccionario_ingredientes[ingrediente['ingrediente']] = ingrediente['gramos']

diccionario_ingredientes


{'arroz': '200', 'azúcar': '150', 'mantequilla': '50'}

In [25]:
# Alternativamente

diccionario = {x['ingrediente']:x['gramos'] for x in matches}
diccionario

{'arroz': '200', 'azúcar': '150', 'mantequilla': '50'}

### Ejercicio 3
Crea una lista con todos los valores de `span()` de los patrones encontrados en `matches`, almacenados como tuplas.

In [26]:
#solución

[matches[i].span() for i in range(len(matches))]

[(35, 54), (57, 77), (164, 188)]

### Ejercicio 4

Dado el siguente texto:

In [27]:
texto = 'Some authors like Jason Foster (y.foster@abcd.com), R. Davis (rdavis22@www.uk) and Charlotte Williams (ch_williams@usa.gov) observed that...'

#### Ej 4.1
Extrae todos los nombres y apellidos de personas de este texto como una lista de diccionarios con las claves `nombre` y `apellido`. Convierte esa lista en un DataFrame de Pandas.

In [28]:
#Solución
import pandas as pd
matches = re.findall(r'([A-Z]+[.a-z]+) ([A-Z][a-z]+)', texto)

pd.DataFrame([{'nombre':matches[i][0],'apellido':matches[i][1]} for i in range(len(matches))])


,nombre,apellido
0,Jason,Foster
1,R.,Davis
2,Charlotte,Williams


In [36]:
# Alternativo

matches = re.finditer(r'(?P<nombre>[A-Z]+[.a-z]+) (?P<apellido>[A-Z][a-z]+)', texto)

pd.DataFrame([{'nombre':x['nombre'],'apellido':x['apellido']} for x in matches])

,nombre,apellido
0,Jason,Foster
1,R.,Davis
2,Charlotte,Williams


#### Ej. 4.2
Extrae todos los emails del texto anterior como una lista.

In [37]:
#Solución
matches = re.findall(r'[\w.a-z\d]+@[a-z]+.[a-z]+', texto)
matches

['y.foster@abcd.com', 'rdavis22@www.uk', 'ch_williams@usa.gov']

## Uso de non-capturing groups
Cuando usamos un grupo para definir un patrón complejo pero queremos capturar toda la expresión usamos un *non-capturing group*:

In [98]:
texto = "entre el 15 de agosto y el 20 de septiembre, no el 15 de 2020"
re.findall(r'\d+ de (?:enero|febrero|marzo|abril|mayo|junio|julio|agosto|\w+bre\b)', texto)

['15 de agosto', '20 de septiembre']

### Ejercicio 5
Crea un patrón RegEx para detectar todas las URL que pertenezcan a un dominio `.es` del siguiente texto:  
Ayuda: el patrón debe buscar el texto `.es` detrás de un patrón repetido `algo.` (una o más veces), seguido de un patrón opcional de tipo `/algo/algo_más`

In [38]:
texto = "amazon.com amazon.es google.es/shopping aliexpress.com.es www.elcorteingles.es"

In [39]:
#Solución
re.findall(r'[.\w]+es[\/a-z]*', texto)

['amazon.es',
 'google.es/shopping',
 'aliexpress.com.es',
 'www.elcorteingles.es']

## Substitución de patrones
La función `re.sub()` permite sustituir texto capturado por una expresión regular.

### Ejercicio 6
Sustituye en el siguiente texto los importes expresados como `$valor` a la forma `valor$`  
Nota: el símbolo `$` tiene un significado especial en RegEx por lo que hay que escaparlo como `\$`

In [40]:
texto = "El coste total fue de $320, repartidos en $225.7 en comida y $94.3 en bebida"



In [41]:
#solución

texto_sustituido = re.sub(r'\$(\d+(?:\.\d+)?)', r'\1$', texto)

print(texto_sustituido)

El coste total fue de 320$, repartidos en 225.7$ en comida y 94.3$ en bebida


## Uso de RegEx con objetos de Pandas
Si tenemos un objeto Series de Pandas podemos aplicar las funciones de texto o de búsqueda de expresiones regulares sobre el contenido de cada elemento (en un DataFrame cada columna es un objeto Series).

In [43]:
import pandas as pd

frases = ["Tengo cita con el doctor a las 2:45.", 
          "el martes llegaré a las 11:30.",
          "No puedo, tengo un partido a las 7:00.",
          "Nos vemos el jueves 7 a las 8:30.",
          "El tren sale a las 9:15 y llega a las 11:35."]

df = pd.DataFrame(frases, columns=['texto'])
df

,texto
0,Tengo cita con el doctor a las 2:45.
1,el martes llegaré a las 11:30.
2,"No puedo, tengo un partido a las 7:00."
3,Nos vemos el jueves 7 a las 8:30.
4,El tren sale a las 9:15 y llega a las 11:35.


### Captura de grupos:

In [44]:
# busca las ocurrencias de hora:minuto en los textos
df['texto'].str.findall(r'(\d{1,2}):(\d{2})')

0              [(2, 45)]
1             [(11, 30)]
2              [(7, 00)]
3              [(8, 30)]
4    [(9, 15), (11, 35)]
Name: texto, dtype: object

### Captura de grupos numerados:

In [45]:
# extrae todas las apariciones de tiempo y separa horas y minutos
df['texto'].str.extractall(r'(?P<tiempo>(?P<hora>\d?\d):(?P<minutos>\d\d))')

tiempo hora minutos
  match                    
0 0       2:45    2      45
1 0      11:30   11      30
2 0       7:00    7      00
3 0       8:30    8      30
4 0       9:15    9      15
  1      11:35   11      35

### Ejercicio 7
Extrae todas las fechas de los siguientes textos, primero de manera completa y luego separando día, mes y año

In [46]:
fechas = ["Tengo cita con el doctor el 3/10", 
          "Juan nació el 28/3/78",
          "Su primer hijo nació el nació el 3-10-2001 y el segundo el 10-1-2003",
          "El 8/1/1998 se fué de viaje a Praga"]

df = pd.DataFrame(fechas, columns=['texto'])
df

,texto
0,Tengo cita con el doctor el 3/10
1,Juan nació el 28/3/78
2,Su primer hijo nació el nació el 3-10-2001 y e...
3,El 8/1/1998 se fué de viaje a Praga


In [47]:
# Solución

df['texto'].str.extractall(r'(\d+[/-]\d+[/-]?\d?\d?\d?\d?)')



0
  match           
0 0           3/10
1 0        28/3/78
2 0      3-10-2001
  1      10-1-2003
3 0       8/1/1998

In [48]:
df['texto'].str.extractall(r'(?P<dia>\d+)[/-](?P<mes>\d+)[/-]?(?P<año>\d?\d?\d?\d?)')


dia mes   año
  match              
0 0       3  10   NaN
1 0      28   3    78
2 0       3  10  2001
  1      10   1  2003
3 0       8   1  1998

### Ejercicio 8
A partir de la siguiente lista de Tweets, genera un DataFrame con las siguientes columnas:  
 - `Tweet`: texto del tweet
 - `Menciones`: lista con las menciones de cada tweet
 - `Hashtag`: lista con los hashtags de cada tweet

In [49]:
tweets = [
    '@Yulian_Poe @guillermoterry1 Ah. mucho más por supuesto! solo que lo incluyo. Me habías entendido mal',
    'Se ha terminado #Rio2016 Lamentablemente no arriendo las ganancias al pueblo brasileño por la penuria que les espera',
    '@Yosmath @Planeta87Radio  #Incantus  Genial ya estoy conectada',
    '@seestrena #seestrenasietevidas Sería un gato tipo Garfield porqué soy un poco vago y porqué me encanta la lasaña!!',
    'Hoy toca escuchar a #MiguelRios con otros grandes artistas...  #Insurrección...'
]

In [50]:
#solucion

df = pd.DataFrame(tweets, columns=['tweets'])
print(df)

df_new = df['tweets'].str.extractall(r'(?P<Tweet>.+)').reset_index(drop=True)
df_new['Menciones'] = df['tweets'].str.findall(r'(?P<Menciones>@\w+)')
df_new['Hashtag'] =  df['tweets'].str.findall(r'(?P<Menciones>#\w+)')

print(df_new)


                                              tweets
0  @Yulian_Poe @guillermoterry1 Ah. mucho más por...
1  Se ha terminado #Rio2016 Lamentablemente no ar...
2  @Yosmath @Planeta87Radio  #Incantus  Genial ya...
3  @seestrena #seestrenasietevidas Sería un gato ...
4  Hoy toca escuchar a #MiguelRios con otros gran...
                                               Tweet  \
0  @Yulian_Poe @guillermoterry1 Ah. mucho más por...   
1  Se ha terminado #Rio2016 Lamentablemente no ar...   
2  @Yosmath @Planeta87Radio  #Incantus  Genial ya...   
3  @seestrena #seestrenasietevidas Sería un gato ...   
4  Hoy toca escuchar a #MiguelRios con otros gran...   

                         Menciones                       Hashtag  
0  [@Yulian_Poe, @guillermoterry1]                            []  
1                               []                    [#Rio2016]  
2      [@Yosmath, @Planeta87Radio]                   [#Incantus]  
3                     [@seestrena]        [#seestrenasietevidas]  
4         

### Ejercicio 9
A partir de la siguiente receta genera un dataframe con las siguientes columnas:
- `Ingrediente`: nombre del ingrediente
- `Cantidad`: valor numérico de la cantidad
- `Unidades`: unidad en la que se expresa la cantidad  
Nota: `Cantidad` y `Unidades` son opcionales. Considera como posibles unidades 'gr', 'gramos', 'litro', 'litros','l' y 'ml'

In [51]:
texto = """
200 gr de arroz redondo.
1 litro de leche entera.
100 gr de azúcar.
2 ramas de canela.
La cáscara de un limón.
2 ml de esencia de vainilla.
50 gramos de mantequilla (Opcional).
"""

In [61]:
#Solución


patron = r'(\d+(?:\.\d+)?)?\s*(gr|gramos|litro|litros|l|ml)?\s+de\s+(\w[\w\s]*)'

# Extraer los datos de cada línea de la receta
datos = [re.findall(patron, linea.strip())[0] for linea in texto.split('\n') if linea.strip()]

# Crear el dataframe
df = pd.DataFrame(datos, columns=['Cantidad', 'Unidades', 'Ingrediente'])

# Convertir la columna 'Cantidad' a tipo numérico y llenar los valores nulos con 1
df['Cantidad'] = pd.to_numeric(df['Cantidad'].fillna(1))

# Mapear las diferentes unidades a los valores deseados
unidades_dict = {'gr': 'gramos', 'l': 'litros'}
df['Unidades'] = df['Unidades'].map(unidades_dict).fillna(df['Unidades'])

# Imprimir el dataframe
print(df)









   Cantidad Unidades          Ingrediente
0     200.0   gramos        arroz redondo
1       1.0    litro         leche entera
2     100.0   gramos               azúcar
3       NaN                        canela
4       NaN                      un limón
5       2.0       ml  esencia de vainilla
6      50.0   gramos         mantequilla 
